# Baseline: Arrhenius Kinetics + TTA + Bagged MLP

Implementing the top-performing approach from the seed prompt:
1. Arrhenius kinetics features (1/T, ln(time))
2. Test Time Augmentation for chemical symmetry
3. Bagged ensemble of 7 MLPs with HuberLoss
4. Spange descriptors for solvent features

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.set_default_dtype(torch.double)
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]

GPU available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB


In [2]:
# Load data functions
def load_data(name="full"):
    if name == "full":
        df = pd.read_csv('/home/data/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv('/home/data/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features():
    return pd.read_csv('/home/data/spange_descriptors_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

SPANGE_DF = load_features()
print(f"Spange descriptors shape: {SPANGE_DF.shape}")
print(f"Solvents available: {list(SPANGE_DF.index)}")

Spange descriptors shape: (26, 13)
Solvents available: ['Cyclohexane', 'Ethyl Acetate', 'Acetic Acid', '2-Methyltetrahydrofuran [2-MeTHF]', '1,1,1,3,3,3-Hexafluoropropan-2-ol', 'IPA [Propan-2-ol]', 'Ethanol', 'Methanol', 'Ethylene Glycol [1,2-Ethanediol]', 'Acetonitrile', 'Water', 'Diethyl Ether [Ether]', 'MTBE [tert-Butylmethylether]', 'Dimethyl Carbonate', 'tert-Butanol [2-Methylpropan-2-ol]', 'DMA [N,N-Dimethylacetamide]', '2,2,2-Trifluoroethanol', 'Dihydrolevoglucosenone (Cyrene)', 'Decanol', 'Butanone [MEK]', 'Ethyl Lactate', 'Methyl Propionate', 'THF [Tetrahydrofuran]', 'Water.Acetonitrile', 'Acetonitrile.Acetic Acid', 'Water.2,2,2-Trifluoroethanol']


In [3]:
# Kinetic Featurizer with Arrhenius features
class KineticMixingFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # 2 numeric + 3 kinetic + 13 spange = 18 features
        self.feats_dim = self.featurizer.shape[1] + 2 + 3

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # --- ARRHENIUS KINETIC ENGINEERING ---
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (Arrhenius)
        log_time = np.log(time_m + 1e-6)  # Log of time
        interaction = inv_temp * log_time  # Interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # --- SYMMETRIC CHEMICAL FEATURES ---
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # SYMMETRY FLIP: Swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

In [4]:
# MLP Architecture
class MLPInternal(nn.Module):
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid()  # Output in [0, 1]
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Symmetric Bagged Model
class SymmetricBaggedModel(nn.Module):
    def __init__(self, data='single', n_models=7, epochs=300):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.epochs = epochs
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, verbose=False):
        # 1. Standard Data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # 2. Augmented Data (Train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i)  # Different seed for each model
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss()  # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(self.epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        # --- TEST TIME AUGMENTATION (TTA) ---
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
            
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

In [ ]:
# Test on one fold first to verify everything works
print("Testing on one fold...")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
(train_X, train_Y), (test_X, test_Y) = next(split_generator)

print(f"Train size: {len(train_X)}, Test size: {len(test_X)}")

# Quick test with fewer models and epochs
model = SymmetricBaggedModel(data='single', n_models=2, epochs=50)
model.train_model(train_X, train_Y)
predictions = model.predict(test_X)

print(f"Predictions shape: {predictions.shape}")
print(f"Sample predictions: {predictions[:3]}")
print(f"Actual values: {test_Y.values[:3]}")

In [ ]:
# Full CV evaluation
import time

def run_cv_evaluation(n_models=7, epochs=300):
    all_predictions = []
    all_actuals = []
    
    # Task 0: Single Solvent (24 folds)
    print("\n--- TASK 0: Single Solvent (Leave-One-Solvent-Out) ---")
    X, Y = load_data("single_solvent")
    split_generator = generate_leave_one_out_splits(X, Y)
    
    task0_preds = []
    task0_actuals = []
    
    start_time = time.time()
    for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = SymmetricBaggedModel(data='single', n_models=n_models, epochs=epochs)
        model.train_model(train_X, train_Y)
        predictions = model.predict(test_X).numpy()
        
        task0_preds.append(predictions)
        task0_actuals.append(test_Y.values)
        
        for row_idx, row in enumerate(predictions):
            all_predictions.append({
                "task": 0, "fold": fold_idx, "row": row_idx,
                "target_1": row[0], "target_2": row[1], "target_3": row[2]
            })
    
    task0_time = time.time() - start_time
    print(f"Task 0 completed in {task0_time/60:.1f} minutes")
    
    # Calculate Task 0 MSE
    task0_preds_all = np.vstack(task0_preds)
    task0_actuals_all = np.vstack(task0_actuals)
    task0_mse = np.mean((task0_preds_all - task0_actuals_all) ** 2)
    print(f"Task 0 MSE: {task0_mse:.6f}")
    
    # Task 1: Full Data (Leave-One-Ramp-Out)
    print("\n--- TASK 1: Full Data (Leave-One-Ramp-Out) ---")
    X, Y = load_data("full")
    split_generator = generate_leave_one_ramp_out_splits(X, Y)
    
    # Count number of ramps
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    n_ramps = len(ramps)
    print(f"Number of ramps: {n_ramps}")
    
    task1_preds = []
    task1_actuals = []
    
    start_time = time.time()
    for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=n_ramps):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = SymmetricBaggedModel(data='full', n_models=n_models, epochs=epochs)
        model.train_model(train_X, train_Y)
        predictions = model.predict(test_X).numpy()
        
        task1_preds.append(predictions)
        task1_actuals.append(test_Y.values)
        
        for row_idx, row in enumerate(predictions):
            all_predictions.append({
                "task": 1, "fold": fold_idx, "row": row_idx,
                "target_1": row[0], "target_2": row[1], "target_3": row[2]
            })
    
    task1_time = time.time() - start_time
    print(f"Task 1 completed in {task1_time/60:.1f} minutes")
    
    # Calculate Task 1 MSE
    task1_preds_all = np.vstack(task1_preds)
    task1_actuals_all = np.vstack(task1_actuals)
    task1_mse = np.mean((task1_preds_all - task1_actuals_all) ** 2)
    print(f"Task 1 MSE: {task1_mse:.6f}")
    
    # Overall MSE
    all_preds = np.vstack([task0_preds_all, task1_preds_all])
    all_actuals = np.vstack([task0_actuals_all, task1_actuals_all])
    overall_mse = np.mean((all_preds - all_actuals) ** 2)
    print(f"\nOverall MSE: {overall_mse:.6f}")
    
    return all_predictions, overall_mse, task0_mse, task1_mse

In [ ]:
# Run full evaluation with reduced settings for faster iteration
# Using 5 models and 200 epochs for faster training
all_predictions, overall_mse, task0_mse, task1_mse = run_cv_evaluation(n_models=5, epochs=200)

In [ ]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)

submission = pd.DataFrame(all_predictions)
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f"Submission saved with {len(submission)} rows")
print(f"\nFinal CV Score (MSE): {overall_mse:.6f}")
print(f"Task 0 MSE: {task0_mse:.6f}")
print(f"Task 1 MSE: {task1_mse:.6f}")